<a href="https://colab.research.google.com/github/yorkjong/stock-reports/blob/main/notebooks/weinstein_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Screening Inspired by Stan Weinstein

This notebook features stock screening methods inspired by Stan Weinstein's book, *Secrets for Profiting in Bull and Bear Markets*. The strategies and criteria used in this analysis are based on the principles outlined in Weinstein's work, aiming to identify potential investment opportunities in both bull and bear markets.


## Install and Setup (this section will be executed automatically)

#### Install Required Packages

In [36]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ranking_utils"
%pip install requests-cache

!wget -O NotoSansTC-Regular.ttf https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ranking_utils) to /tmp/pip-req-build-9binh9pn
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-9binh9pn
  Running command git checkout -b feature/ranking_utils --track origin/feature/ranking_utils
  Switched to a new branch 'feature/ranking_utils'
  Branch 'feature/ranking_utils' set up to track remote branch 'feature/ranking_utils' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 2aed48fc5c43e0b3761fa04e0b9daebef7207dee
  Preparing metadata (setup.py) ... done
--2024-10-13 09:21:40--  https://share.cole.tw/d/Tools%20-%20MAC/Fonts/Noto_Sans_TC/static/NotoSansTC-Regular.ttf?sign=bATsZP5QZdI_2EM15sAbcAE48Cacle91CpwUNOCMuM8=:0
Resolving share.cole.tw (share.cole.tw)... 180.218.0.193
Connecting to share.cole.tw (share.cole.tw)|180.218.0.193|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7110560 (6.8M) [

### Setup and Configuration

In [37]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [38]:
# @title Set Chinese Font for matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

fm.fontManager.addfont('NotoSansTC-Regular.ttf')
font_name = 'Noto Sans TC'
if font_name not in plt.rcParams['font.sans-serif']:
    plt.rcParams['font.sans-serif'].insert(0, font_name)

In [39]:
# @title ReadOnlyGitHub
import requests
import pandas as pd
from io import StringIO

class ReadOnlyGitHub:
    def __init__(self, repo_owner, repo_name, dir='', branch='main'):
        dir = dir.strip('/')
        base = 'https://raw.githubusercontent.com'
        if dir:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}/{dir}'
        else:
            self.raw_url = f'{base}/{repo_owner}/{repo_name}/{branch}'

        base = 'https://api.github.com/repos'
        if dir:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.api_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch

    def file_exists(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        response = requests.head(url)
        return response.status_code == 200

    def list_filenames(self, dir_path=''):
        if dir_path:
            url = f'{self.api_url}/{dir_path}?ref={self.branch}'
        else:
            url = f'{self.api_url}?ref={self.branch}'
        response = requests.get(url)
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        elif response.status_code == 404:
            print(f"Directory not found: {url}")
            return []
        else:
            print(f"Request failed: {response.status_code} - {response.text}")
            return []

    def download_csv(self, file_path):
        url = f'{self.raw_url}/{file_path}'
        if self.file_exists(file_path):
            return pd.read_csv(url)
        else:
            return pd.DataFrame()

# Create a GitHub instance
github = ReadOnlyGitHub(
    repo_owner='yorkjong',
    repo_name='stock-reports',
    dir='weinstein',
    branch='data'
)

In [40]:
# @title Taiwan Stock Name Lookup

class StockNameLookup:
    _df = None  # Class-level variable to hold the DataFrame

    @classmethod
    def _load_data(cls):
        if cls._df is None:  # Check if the DataFrame is already loaded
            gh = ReadOnlyGitHub(
                repo_owner='yorkjong',
                repo_name='stock-reports',
                dir='data/stock_list',
            )
            cls._df = gh.download_csv('taiwan_stock_OpenAPI.csv')

    @classmethod
    def tw_stock_name(cls, ticker):
        cls._load_data()  # Ensure data is loaded before accessing

        code = ticker.split('.')[0]  # Extract the code part

        # Filter the DataFrame to find the stock name for the given code
        stock_name = cls._df.loc[cls._df['Code'] == code, 'Name']

        # Check if the stock_name is empty and return an appropriate message
        if not stock_name.empty:
            return stock_name.values[0]  # Return the first matched stock name
        else:
            return None  # Return None if ticker not found


def tw_stock_name(ticker):
    return StockNameLookup.tw_stock_name(ticker)

In [41]:
# @title Metadata

def parse_metadata(filename):
    components = filename.split('_')
    if len(components) != 5:
        raise ValueError("Filename does not have the expected number of components")

    source, kind, period, ma, date = components
    date = date.replace('.csv', '')

    return {
        "Source": source,
        "Type": kind,
        "Period": period,
        "MA": ma,
        "Date": date
    }


def print_metadata(meatadata):
    for key, value in meatadata.items():
        print(f"{key}: {value}")

In [42]:
# @title IBD Financial files
import re
from datetime import datetime


def get_latest_file(file_list, source):
    pattern = rf'^{re.escape(source)}_(\w+)_fin_(\d+)\.csv$'
    matching_files = []

    for file in file_list:
        match = re.match(pattern, file)
        if match:
            date_str = match.group(2)
            date = datetime.strptime(date_str, "%Y%m%d")
            matching_files.append((file, date))

    if not matching_files:
        return None

    return max(matching_files, key=lambda x: x[1])[0]


def fin_download_latest_csv(source):
    gh = ReadOnlyGitHub(
        repo_owner='yorkjong',
        repo_name='stock-reports',
        dir='ibd_fin',
        branch='data'
    )
    file_list = gh.list_filenames()
    fin_filename = get_latest_file(file_list, source)
    df = gh.download_csv(fin_filename)
    return df, fin_filename

In [43]:
# @title DataFrame Utilities

def print_column(df, column):
    if column in df.columns:
        print(', '.join(df[column]))

In [44]:
# @title Source of Tickers

def tickers_from_df(df):
    if 'Name' in df.columns:
        return [name.strip() for names in df['Name']
                for name in names.split(',')]
    elif 'Ticker' in df.columns:
        return [ticker.strip() for tickers in df['Ticker']
                for ticker in tickers.split(',')]
    return []

def major_indices():
    return ['^DJI', '^IXIC', '^NDX', '^RUT', '^SOX',
            '^TWII', '^N225', '^HSI',
            '^STOXX50E', '^FTSE', '^GDAXI', '^FCHI', '^GSPTSE']

def sector_indices():
    return ['SOXX', 'DVY',
            'IWB','IWM', 'IWV',  'IJR',
            'ITB', 'IHI', 'IYC', 'ITA', 'IAK',
            'IYZ', 'IYT', 'IYR', 'IYF', 'IYJ',
            'IYG', 'IYH', 'IYK', 'IDU', 'IYE', 'IHE',
            'IAT', 'IAI', 'IEO', 'IYM', 'IHF']

In [45]:
# @title Checkboxes

import ipywidgets as widgets

def cbs_create(symbols, n_pre_checked=10):
    '''Create a list of checkboxes'''
    return [
        widgets.Checkbox(
            value=(i < n_pre_checked),  # Set first n items as checked
            description=symbol,
            layout=widgets.Layout(width='auto'),
            style={'description_width': 'auto'}
        )
        for i, symbol in enumerate(symbols)
    ]

def cbs_with_grid(checkboxes, n_cols=5):
    '''Create a grid layouting the given checkboxes'''
    return widgets.GridBox(checkboxes, layout=widgets.Layout(
        width='auto',
        grid_template_columns=f'repeat({n_cols}, 1fr)',
        grid_gap='10px'  # Add some space between the checkboxes
    ))

def cbs_get_selected(checkboxes):
    '''Get the selected symbols from the given checkboxes'''
    return [checkbox.description for checkbox in checkboxes if checkbox.value]

def cbs_unselect_all(checkboxes):
    '''Unselect all checkboxes in the given list'''
    for checkbox in checkboxes:
        checkbox.value = False

def cbs_select_top(checkboxes, n=10):
    '''Select the top n checkboxes in the given list'''
    for i, checkbox in enumerate(checkboxes):
        checkbox.value = (i < n)

In [46]:
# @title Dropdown Menus

def create_period_dropdown(value='2y'):
    return widgets.Dropdown(
        options=['1y', '2y', '5y'],
        value=value,
        description='Period:',
    )

def create_interval_dropdown(value='1wk'):
    return widgets.Dropdown(
        options=['1d', '1wk'],
        value=value,
        description='Interval:',
    )

def create_style_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['default', 'classic', 'yahoo', 'charles', 'tradingview', 'binance', 'binancedark', 'mike', 'nightclouds', 'checkers', 'ibd', 'sas', 'starsandstripes', 'kenan', 'blueskies', 'brasil'],
        value='yahoo' if value is None else value,
        description='Style:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

def create_template_dropdown(desc=None, value=None):
    return widgets.Dropdown(
        options=['plotly', 'plotly_white', 'plotly_dark', 'ggplot2', 'seaborn', 'simple_white', 'presentation', 'xgridoff', 'ygridoff'],
        value='plotly_dark' if value is None else value,
        description='Template:' if desc is None else desc,
        style={'description_width': 'initial'},
    )

In [47]:
# @title Multiple Searchable Dropdown Menus
import ipywidgets as widgets

def create_search_box():
    '''Create a Text widget for search input'''
    return widgets.Text(
        description='Search:',
        placeholder='Type to search',
        layout=widgets.Layout(width='auto')
    )

def create_dropdown(options, description='Stock:'):
    '''Create a Dropdown widget for displaying filtered options'''
    return widgets.Dropdown(
        description=description,
        options=[None] + options,  # None as the default option
        layout=widgets.Layout(width='auto'),
        value=None  # Set default value to None
    )

def update_dropdown(change, dropdown, options):
    '''Update the options in the dropdown based on search input'''
    search_text = change['new'].lower()
    filtered = [option for option in options if search_text in option.lower()]
    if filtered:
        dropdown.options = [None] + filtered
        dropdown.value = filtered[0]  # Auto-select the first matching option
    else:
        dropdown.options = [None]  # Retain only the None option if no match

def remove_duplicates_preserve_order(lst):
    '''Remove duplicates from a list while preserving order'''
    seen = set()
    result = []
    for item in lst:
        if item and item not in seen:
            seen.add(item)
            result.append(item)
    return result

def get_dropdowns_selected_options(dropdowns):
    '''Get selected options from the dropdowns'''
    selected = [dropdown.value for dropdown in dropdowns if dropdown.value]
    return remove_duplicates_preserve_order(selected)

def create_search_dropdowns(options, max_selections):
    '''Create a layout with search boxes and dropdowns'''
    # Create UI components
    search_boxes = [create_search_box() for _ in range(max_selections)]
    dropdowns = [create_dropdown(options) for _ in range(max_selections)]

    # Bind search box and dropdown menu events
    for search_box, dropdown in zip(search_boxes, dropdowns):
        search_box.observe(lambda change, dropdown=dropdown:
                            update_dropdown(change, dropdown, options),
                            names='value')
    # Create the layout
    controls = [widgets.HBox([search_box, dropdown])
                for search_box, dropdown  in zip(search_boxes, dropdowns)]
    layout = widgets.VBox(controls)
    return dropdowns, layout

In [48]:
# @title Outputs
outputs = widgets.VBox()

In [49]:
# @title enable_plotly_in_cell
# ref. https://stackoverflow.com/questions/76593068/plotly-figure-not-rendering-in-ipywidgets-interact-function-google-colab
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

### Glossary of Terms

#### Source (The Source of Stocks to Analyze)
- **Description**: Stocks can be sourced from various exchanges or indices.
- **Common Abbreviations**:
  - **Taiwan Markets**:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所)
    - `TPEX`: Taipei Exchange (上櫃交易所)
    - `ESB`: Emerging Stock Board (興櫃交易所)
    - Combine with `+` (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - **America Markets**:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `RUI`: Russell 1000
    - `RUT`: Russell 2000
    - `SOX`: PHLX Semiconductor Index (費半指數)
    - `W5000`: Wilshire 5000 Total Market Index
    - `USLS`: U.S. Listed Stocks (美國上市股票)
    - `U.S.Listed`: U.S. Listed Stocks (美國上市股票)
    - Combine with `+` (e.g., `SPX+DJIA+NDX+SOX`)

#### Period (Historical Data Time Range)
- **Description**: The duration for fetching historical data.
- **Example**: `2y` for 2 years

#### Interval (Historical Data Frequency)
- **Description**: The frequency of historical data points.
- **Common Intervals**:
  - `'1d'`: Daily data
  - `'1wk'`: Weekly data
  - `'1mo'`: Monthly data

#### MA (Moving Average)
- **SMA**: Simple Moving Average
- **EMA**: Exponential Moving Average

#### RS (Relative Strength)
- **Description**: Measures a stock's performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the performance of the benchmark index or market.
  - A positive value indicates outperformance relative to the benchmark.
  - A negative value indicates underperformance relative to the benchmark.

#### EPS RS (EPS Relative Strength)
- **Description**: Measures a stock's earnings per share (EPS) performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the EPS performance of the benchmark index.
  - A positive value indicates that the stock's EPS growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index EPS Calculation**: Weighted according to component stocks' share counts.

#### Rev RS (Operating Revenue Relative Strength)
- **Description**: Measures a stock's operating revenue performance relative to a benchmark index.
- **Unit**: Percentage (%)
- **Interpretation**:
  - A value of 0 represents the revenue performance of the benchmark index.
  - A positive value indicates that the stock's revenue growth has outperformed the benchmark.
  - A negative value indicates underperformance relative to the benchmark.
- **Index Revenue Calculation**: Weighted according to component stocks' market capitalization.

#### TTM EPS (Trailing Twelve Months EPS)
- **Description**: The earnings per share over the trailing twelve months, which gives a more recent view of the company's profitability.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM EPS indicates better profitability over the past year.
  - Used to compare with previous periods or with other companies.

#### TTM RPS (Trailing Twelve Months Revenue Per Share)
- **Description**: The revenue per share over the trailing twelve months, showing how much revenue a company generates per share.
- **Unit**: Currency per share
- **Interpretation**:
  - A higher TTM RPS indicates better revenue generation capability per share over the past year.
  - Useful for comparing revenue performance with previous periods or with other companies.

#### TTM PE (Trailing Twelve Months PE)
- **Description**: The price-to-earnings ratio calculated using the earnings from the trailing twelve months. It provides a measure of how much investors are willing to pay per dollar of earnings.
- **Unit**: Ratio
- **Interpretation**:
  - A higher TTM PE indicates that investors expect higher future growth and are willing to pay more for the stock.
  - A lower TTM PE might suggest undervaluation or lower expected growth.


##### References:
- [Mansfield Relative Strength (Original Version) by stageanalysis — Indicator by Stage_Analysis — TradingView](https://www.tradingview.com/script/NzUBDDtb-Mansfield-Relative-Strength-Original-Version-by-stageanalysis/)
- [Mansfield Relative Strength | TrendSpider Store](https://trendspider.com/trading-tools-store/indicators/mansfield-relative-strength/)
- [Mansfield Relative Strength | ChartMill.com](https://www.chartmill.com/documentation/technical-analysis/indicators/35-Mansfield-Relative-Strength)
- [How to create the Mansfield Relative Performance Indicator - Stage Analysis](https://www.stageanalysis.net/blog/4266/how-to-create-the-mansfield-relative-performance-indicator)
  - [Stan Weinstein's Stage Analysis | Page 49 | Trade2Win Forums • UK Financial Trading Community](https://www.trade2win.com/threads/stan-weinsteins-stage-analysis.134944/page-49#post-2137398)


# Execute Actions Step by Step

## Step 1. Select and Preivew a File

In [ ]:
# @title Step 1.1 Select a File

import ipywidgets as widgets

with requests_cache.disabled():
    filenames = github.list_filenames()

# Extract and sort all unique values in reverse order
all_dates = sorted(set(fn.split('_')[4].replace('.csv', '') for fn in filenames), reverse=True)
all_sources = sorted(set(fn.split('_')[0] for fn in filenames))
all_types = sorted(set(fn.split('_')[1] for fn in filenames), reverse=True)
all_periods = sorted(set(fn.split('_')[2] for fn in filenames), reverse=True)

# Calculate the maximum length of filenames to set dropdown width
max_filename_length = max(len(fn) for fn in filenames)
dropdown_width = f'{max_filename_length * 10}px'  # 10px width per character

# Create dropdowns with dynamic width
date_dropdown = widgets.Dropdown(
    options=all_dates,
    description='Date:',
    layout=widgets.Layout(width=dropdown_width)
)

source_dropdown = widgets.Dropdown(
    options=all_sources,
    description='Source:',
    layout=widgets.Layout(width=dropdown_width)
)

type_dropdown = widgets.Dropdown(
    options=all_types,
    description='Type:',
    layout=widgets.Layout(width=dropdown_width)
)

period_dropdown = widgets.Dropdown(
    options=all_periods,
    description='Period:',
    layout=widgets.Layout(width=dropdown_width)
)

# Create file selection dropdown
file_dropdown = widgets.Dropdown(
    options=[],
    description='File:',
    layout=widgets.Layout(width=dropdown_width)
)

output = widgets.Output()

def update_dropdowns(*args):
    # Filter files based on selected date
    date_filtered_files = [fn for fn in filenames if date_dropdown.value in fn]

    # Update Source dropdown
    available_sources = sorted(set(fn.split('_')[0] for fn in date_filtered_files))
    source_dropdown.options = available_sources
    if source_dropdown.value not in available_sources:
        source_dropdown.value = available_sources[0] if available_sources else None

    # Update Type dropdown
    available_types = sorted(set(fn.split('_')[1] for fn in date_filtered_files), reverse=True)
    type_dropdown.options = available_types
    if type_dropdown.value not in available_types:
        type_dropdown.value = available_types[0] if available_types else None

    # Update Period dropdown
    available_periods = sorted(set(fn.split('_')[2] for fn in date_filtered_files), reverse=True)
    period_dropdown.options = available_periods
    if period_dropdown.value not in available_periods:
        period_dropdown.value = available_periods[0] if available_periods else None

    # Update file options
    update_file_options()

def update_file_options(*args):
    filtered_files = [
        fn for fn in filenames
        if (date_dropdown.value in fn and
            source_dropdown.value == fn.split('_')[0] and
            type_dropdown.value == fn.split('_')[1] and
            period_dropdown.value == fn.split('_')[2])
    ]
    file_dropdown.options = filtered_files
    if filtered_files:
        global selected_file, metadata
        file_dropdown.value = filtered_files[0]  # Set initial value to the first match
        selected_file = file_dropdown.value
        metadata = parse_metadata(selected_file)
        with output:
            output.clear_output()
            print_metadata(metadata)
    else:
        file_dropdown.value = None

# Bind event handlers
date_dropdown.observe(update_dropdowns, 'value')
source_dropdown.observe(update_file_options, 'value')
type_dropdown.observe(update_file_options, 'value')
period_dropdown.observe(update_file_options, 'value')

# Display all dropdowns
display(date_dropdown, source_dropdown, type_dropdown, period_dropdown,
        file_dropdown, output)

# Initialize dropdowns
update_dropdowns()

Dropdown(description='Date:', layout=Layout(width='370px'), options=('20241012', '20241010'), value='20241012'…

Dropdown(description='Source:', layout=Layout(width='370px'), options=('SPX', 'TWSE+TPEX', 'U.S.Listed'), valu…

Dropdown(description='Type:', layout=Layout(width='370px'), options=('stocks',), value='stocks')

Dropdown(description='Period:', layout=Layout(width='370px'), options=('2y',), value='2y')

Dropdown(description='File:', layout=Layout(width='370px'), options=(), value=None)

Output()

In [26]:
# @title Step 1.2 Load and Preview the File
print(f'{selected_file}:')
with requests_cache.disabled():
    df_rs = github.download_csv(selected_file)
display(df_rs)

with requests_cache.disabled():
    df_fin, fn_fin = fin_download_latest_csv(metadata['Source'])
print(f'\n{fn_fin}:')
display(df_fin)

print(f'\nMerged:')
df_rs2merge = df_rs.drop(['Rating (1M)', 'Rating (3M)', 'Rating (6M)',
                          'Rating (9M)'],
                         axis=1)

columns_to_keep = ['Ticker', 'Sector', 'Industry',
                   'EPS RS', 'TTM EPS', 'Rev RS', 'TTM RPS', 'TTM PE']
df_fin2merge = df_fin[columns_to_keep]

df_merge = pd.merge(df_rs2merge, df_fin2merge, on='Ticker', how='left')
display(df_merge)

U.S.Listed_stocks_2y_SMA_20241013.csv:


,Ticker,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Rating (1M),Rating (3M),Rating (6M),Rating (9M),Price,52W pos,MA10,MA30,Volume / VMA10
0,CHSN,348.43,481.89,-24.71,2.49,-26.54,-50.94,99.0,28.0,84.0,25.0,10.0,12.24,0.81,4.27,2.77,2.55
1,DOGZ,226.47,190.86,129.74,68.86,-30.16,-69.65,99.0,99.0,98.0,22.0,5.0,40.24,1.00,25.67,16.18,1.96
2,WGS,219.74,184.69,140.78,201.50,62.82,-65.44,99.0,99.0,99.0,97.0,6.0,60.00,1.00,39.89,27.48,2.00
3,LASE,218.22,180.02,225.25,0.59,-36.12,-64.91,99.0,99.0,83.0,18.0,6.0,8.53,0.57,6.47,3.47,6.31
4,CAPR,201.83,289.55,-16.89,-13.33,28.98,12.72,99.0,37.0,55.0,93.0,87.0,17.77,0.80,7.99,6.24,1.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5441,ZJK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5442,ZK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5443,ZKH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5444,ZOOZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



U.S.Listed_stocks_fin_20241010.csv:


,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
0,LUNR,Industrials,Aerospace & Defense,7.1200,9.18,-7.84,-3.79,-4.58,NaN,6.003049e+09,-0.17,83.11,4.810,NaN,99.0,91.0
1,BZFD,Communication Services,Internet Content & Information,2.6900,3.84,-9.17,4.00,2.88,NaN,4.060600e+09,-1.12,-36.62,6.229,NaN,99.0,9.0
2,CART,Consumer Cyclical,Internet Retail,43.9900,-1.14,22.69,1.05,22000000.00,47000000.0,2.300000e+09,-5.56,7.22,13.986,NaN,99.0,76.0
3,ACDC,Energy,Oil & Gas Equipment & Services,7.0900,-37.89,55.43,-1.92,-19.82,NaN,1.882714e+09,-1.43,5.76,13.919,NaN,99.0,74.0
4,KNTK,Energy,Oil & Gas Midstream,48.7800,3.50,-13.17,7.10,0.32,NaN,1.644445e+09,2.83,18.60,24.316,17.17,99.0,83.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5437,ZJYL,Healthcare,Medical Instruments & Supplies,2.5550,NaN,NaN,NaN,NaN,NaN,NaN,0.02,NaN,0.129,127.00,NaN,NaN
5438,ZK,Consumer Cyclical,Auto Manufacturers,25.1500,-0.17,-0.35,0.61,NaN,NaN,NaN,-4.99,NaN,314.047,NaN,NaN,NaN
5439,ZKIN,Basic Materials,Steel,0.5050,NaN,NaN,NaN,NaN,NaN,NaN,-1.95,NaN,3.592,NaN,NaN,NaN
5440,ZOOZ,Consumer Cyclical,Specialty Retail,2.3400,NaN,NaN,NaN,NaN,NaN,NaN,-22.44,NaN,0.069,NaN,NaN,NaN



Merged:


,Ticker,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,52W pos,MA10,MA30,Volume / VMA10,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
0,CHSN,348.43,481.89,-24.71,2.49,-26.54,-50.94,99.0,12.24,0.81,4.27,2.77,2.55,Consumer Cyclical,Restaurants,NaN,-0.02,NaN,1.287,NaN
1,DOGZ,226.47,190.86,129.74,68.86,-30.16,-69.65,99.0,40.24,1.00,25.67,16.18,1.96,Consumer Cyclical,Leisure,NaN,-3.62,NaN,1.304,NaN
2,WGS,219.74,184.69,140.78,201.50,62.82,-65.44,99.0,60.00,1.00,39.89,27.48,2.00,Healthcare,Health Information Services,352.93,-4.38,21.98,9.334,NaN
3,LASE,218.22,180.02,225.25,0.59,-36.12,-64.91,99.0,8.53,0.57,6.47,3.47,6.31,Industrials,Specialty Industrial Machinery,1106.15,-0.57,-37.04,0.389,NaN
4,CAPR,201.83,289.55,-16.89,-13.33,28.98,12.72,99.0,17.77,0.80,7.99,6.24,1.51,Healthcare,Biotechnology,165.97,-0.89,189.23,0.907,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5441,ZJK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.53
5442,ZK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consumer Cyclical,Auto Manufacturers,NaN,-4.99,NaN,314.047,NaN
5443,ZKH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consumer Cyclical,Internet Retail,1281.34,-0.37,2.32,82.533,NaN
5444,ZOOZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Consumer Cyclical,Specialty Retail,NaN,-22.44,NaN,0.069,NaN


## Step 2. Filter Stocks

In [23]:
# @title Filter 1. Sorting {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond2 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond3 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond4 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond5 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond6 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond7 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
sorted_column = "RS" # @param ["RS","1 Week Ago","1 Month Ago","3 Months Ago","6 Months Ago","Price","MA10","MA30","Volume / VMA10","EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f1 = df.head(num_items)
display(df_top_f1)

print_column(df_top_f1, 'Name')
print_column(df_top_f1, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
MA: SMA
Date: 20241013


,Ticker,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,52W pos,MA10,MA30,Volume / VMA10,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
45,JVA,79.06,77.80,38.97,51.93,6.22,-38.84,98.0,3.14,0.95,2.58,2.02,0.67,Consumer Defensive,Packaged Foods,1004.75,0.20,1.35,13.333,14.64
50,TIGR,75.36,156.67,-25.08,-5.91,-24.66,0.81,98.0,8.62,0.62,5.13,4.38,5.34,Financial Services,Capital Markets,263.76,0.11,21.84,1.602,73.68
51,CMPO,75.04,74.84,65.27,-1.72,-4.50,-25.01,98.0,14.30,1.00,12.44,8.76,0.89,Industrials,Metal Fabrication,158.64,1.04,0.14,19.384,13.37
52,ADMA,73.75,113.24,106.60,81.83,32.29,8.59,98.0,17.15,0.82,18.04,12.27,1.71,Healthcare,Biotechnology,514.34,0.14,68.02,1.447,121.50
58,CHCI,69.16,78.44,28.09,7.29,3.32,-8.34,98.0,11.39,0.96,8.49,7.11,2.96,Real Estate,Real Estate - Diversified,259.69,0.83,10.09,4.817,12.11
68,GGAL,65.52,49.47,61.44,18.61,47.01,7.83,98.0,49.77,1.00,41.31,34.15,1.12,Financial Services,Banks—Regional,1688.68,2.35,423.28,4001.249,20.45
79,PLTR,61.00,51.87,28.88,20.95,20.17,10.18,98.0,43.51,1.00,34.88,27.50,1.16,Technology,Software—Infrastructure,579.87,0.17,15.25,1.127,256.82
93,LX,56.48,72.97,-14.30,-29.17,-27.38,-30.25,97.0,3.35,0.86,2.14,1.89,2.77,Financial Services,Credit Services,150.29,0.70,2.21,84.354,4.41
115,LWAY,49.36,52.20,30.47,-19.61,71.45,54.28,97.0,26.63,0.99,21.11,18.02,0.47,Consumer Defensive,Packaged Foods,342.89,0.90,10.37,12.028,29.56
150,DAVE,41.55,44.11,33.68,40.63,212.36,14.72,96.0,43.78,0.87,38.44,38.72,1.46,Technology,Software - Application,380.41,2.04,18.20,24.051,20.24


JVA, TIGR, CMPO, ADMA, CHCI, GGAL, PLTR, LX, LWAY, DAVE


In [24]:
# @title Filter 2. Increasing RS > 0 {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['RS'] > df['1 Week Ago'])" # @param ["& (df['RS'] > df['1 Week Ago'])", ""]
cond2 = "& (df['1 Week Ago'] > df['1 Month Ago'])" # @param ["& (df['1 Week Ago'] > df['1 Month Ago'])", "& (df['RS'] > df['1 Month Ago'])", ""]
cond3 = "& (df['1 Month Ago'] > df['3 Months Ago'])" # @param ["& (df['1 Month Ago'] > df['3 Months Ago'])", ""]
cond4 = "& (df['3 Months Ago'] > df['6 Months Ago'])" # @param ["& (df['3 Months Ago'] > df['6 Months Ago'])", ""]
cond5 = "& (df['6 Months Ago'] > df['9 Months Ago'])" # @param ["& (df['6 Months Ago'] > df['9 Months Ago'])", ""]
cond6 = "& (df['Rating (RS)'] > 80)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond8 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond9 = "" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond10 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond11 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond12 = "& (df['Price'] < 100)" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 13
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f2 = df.head(num_items)
display(df_top_f2)

print_column(df_top_f2, 'Name')
print_column(df_top_f2, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
MA: SMA
Date: 20241013


,Ticker,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,52W pos,MA10,MA30,Volume / VMA10,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
51,CMPO,75.04,74.84,65.27,-1.72,-4.50,-25.01,98.0,14.30,1.00,12.44,8.76,0.89,Industrials,Metal Fabrication,1.586400e+02,1.04,0.14,19.384,13.37
79,PLTR,61.00,51.87,28.88,20.95,20.17,10.18,98.0,43.51,1.00,34.88,27.50,1.16,Technology,Software—Infrastructure,5.798700e+02,0.17,15.25,1.127,256.82
161,PAHC,38.92,28.55,27.85,8.63,-13.63,-24.96,96.0,23.49,1.00,20.82,17.76,1.48,Healthcare,Drug Manufacturers—Specialty & Generic,2.667600e+02,0.06,-4.85,25.125,380.75
403,EE,20.56,19.80,-1.77,-9.15,-24.93,-31.93,91.0,23.61,0.62,20.25,18.52,1.22,Energy,Oil & Gas Midstream,5.666669e+08,1.12,-83.45,34.625,20.12
413,MWA,20.07,18.50,12.65,6.71,-4.99,-8.94,91.0,22.73,1.00,20.93,18.80,1.79,Industrials,Specialty Industrial Machinery,2.324700e+02,0.79,-2.45,8.124,27.85
421,UAL,19.75,16.45,-4.07,-10.90,-16.40,-17.50,91.0,61.80,1.00,49.83,49.09,1.13,Industrials,Airlines,2.868300e+02,8.92,11.13,169.490,6.76
445,DTM,18.45,17.59,12.89,4.52,3.42,-4.09,91.0,85.87,1.00,78.14,70.71,1.12,Energy,Oil & Gas Midstream,1.830500e+02,4.15,-3.60,9.918,20.15
448,TSEM,18.21,16.50,15.81,11.56,-12.49,-23.45,91.0,45.82,1.00,42.33,38.54,0.86,Technology,Semiconductors,2.025600e+02,4.44,-11.66,12.536,10.26
476,GLW,17.60,15.36,11.96,3.56,-9.84,-13.38,90.0,46.46,1.00,42.63,38.62,1.43,Technology,Electronic Components,5.920000e+01,0.51,-10.34,14.554,90.30
510,CPNG,16.43,12.85,7.92,2.61,-5.73,-13.07,89.0,25.70,1.00,23.57,21.96,1.30,Consumer Cyclical,Internet Retail,3.551700e+02,0.58,4.85,15.194,44.23


CMPO, PLTR, PAHC, EE, MWA, UAL, DTM, TSEM, GLW, CPNG


In [25]:
# @title Filter 3. RS Breakout {"run":"auto"}
base = 0 # @param [0, 30, 60, 100] {"type":"raw"}
cond0 = "(df['RS'] > base)" # @param ["(df['RS'] > base)"]
cond1 = "" # @param ["& (df['1 Week Ago'] < base)",""]
cond2 = "& (df['1 Month Ago'] < base)" # @param ["& (df['1 Month Ago'] < base)",""]
cond3 = "& (df['3 Months Ago'] < base)" # @param ["& (df['3 Months Ago'] < base)",""]
cond4 = "& (df['6 Months Ago'] < base)" # @param ["& (df['6 Months Ago'] < base)",""]
cond5 = "& (df['9 Months Ago'] < base)" # @param ["& (df['9 Months Ago'] < base)",""]
cond6 = "& (df['Rating (RS)'] > 80)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
cond7 = "& (df['Volume / VMA10'] > 2)" # @param ["& (df['Volume / VMA10'] > 1.5)", "& (df['Volume / VMA10'] > 2)", "& (df['Volume / VMA10'] > 3)", "& (df['Volume / VMA10'] > 4)", ""]
cond8 = "& (df['Price'] > df['MA30'])" # @param ["& (df['Price'] > df['MA10'])", "& (df['Price'] > df['MA30'])", ""]
cond9 = "" # @param ["& (df['MA10'] > df['MA30'])",""]
cond10 = "& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["& ((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond11 = "" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond12 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond13 = "" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond14 = "& (df['52W pos'] > 0.6)" # @param ["& (df['52W pos'] > 0.5)", "& (df['52W pos'] > 0.6)", "& (df['52W pos'] > 0.7)", ""]
cond15 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", "& (df['Price'] < 500)", ""]
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 16
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

print_metadata(metadata)
df = df_merge.copy()

df = df[eval(cond)]
df_top_f3 = df.head(num_items)
display(df_top_f3)

print_column(df_top_f3, 'Name')
print_column(df_top_f3, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
MA: SMA
Date: 20241013


,Ticker,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Price,52W pos,MA10,MA30,Volume / VMA10,Sector,Industry,EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE
93,LX,56.48,72.97,-14.30,-29.17,-27.38,-30.25,97.0,3.35,0.86,2.14,1.89,2.77,Financial Services,Credit Services,150.29,0.70,2.21,84.354,4.41
779,XNET,10.16,20.26,-10.78,-7.62,-19.00,-19.56,84.0,2.02,0.82,1.72,1.68,2.71,Technology,Software—Infrastructure,438.67,0.26,-23.95,4.932,7.62


LX, XNET


In [22]:
# @title Filter 4. Groupby "Industry" {"run":"auto"}
cond0 = "(df['RS'] > 0)" # @param ["(df['RS'] > 0)"]
cond1 = "& (df['Rating (RS)'] > 95)" # @param ["& (df['Rating (RS)'] > 95)", "& (df['Rating (RS)'] > 90)", "& (df['Rating (RS)'] > 85)", "& (df['Rating (RS)'] > 80)", ""]
num_items = 3 # @param [1, 3, 5] {"type":"raw"}

NUM_CONDS = 2

from vistock.ranking_utils import append_ratings, groupby_industry

print_metadata(metadata)
stock_df = df_merge.copy()

# Filter out rows with NaN in the 'Ticker' column
stock_df = stock_df[stock_df['Ticker'].notna()]

rs_columns = ['RS', '1 Week Ago',
              '1 Month Ago', '3 Months Ago', '6 Months Ago', '9 Months Ago']
if 'Name' in stock_df.columns:
    columns = ['Sector', 'Ticker', 'Name'] + rs_columns
else:
    columns = ['Sector', 'Ticker'] + rs_columns
df = groupby_industry(stock_df, columns, key='RS')

df = df.sort_values(by='RS', ascending=False).reset_index(drop=True)
rating_columns = ['Rating (RS)', 'Rating (1W)',
                  'Rating (1M)', 'Rating (3M)', 'Rating (6M)', 'Rating (9M)']
df = append_ratings(df, rs_columns, rating_columns)

cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))
df = df[eval(cond)]
df_top_f4 = df.head(num_items)
display(df_top_f4)

print_column(df_top_f4, 'Name')
print_column(df_top_f4, 'Ticker')

Source: U.S.Listed
Type: stocks
Period: 2y
MA: SMA
Date: 20241013


,Industry,Sector,Ticker,RS,1 Week Ago,1 Month Ago,3 Months Ago,6 Months Ago,9 Months Ago,Rating (RS),Rating (1W),Rating (1M),Rating (3M),Rating (6M),Rating (9M)
0,Real Estate - Diversified,Real Estate,CHCI,69.16,78.44,28.09,7.29,3.32,-8.34,99,99,99,96,89,54
1,Utilities—Independent Power Producers,Utilities,"VST,TLN,PAM,NRG,TAC,KEN",27.58,39.82,17.17,12.56,19.09,8.16,98,98,98,98,98,96
2,Other Precious Metals & Mining,Basic Materials,"PPTA,GATO,EXK,HL,VOXR,MUX,TFPM,BVN,SBSW",16.62,17.39,5.10,10.28,7.70,-11.44,98,97,91,97,93,43


CHCI, VST,TLN,PAM,NRG,TAC,KEN, PPTA,GATO,EXK,HL,VOXR,MUX,TFPM,BVN,SBSW


In [50]:
# @title Filter 5. Financial {"run":"auto"}
cond0 = "((df['EPS RS'] > 0) | (df['Rev RS'] > 0))" # @param ["((df['EPS RS'] > 0) | (df['Rev RS'] > 0))", ""]
cond1 = "& (df['EPS RS'] > 0)" # @param ["& (df['EPS RS'] > 0)", "& (df['EPS RS'] > 50)", "& (df['EPS RS'] > 100)", "& (df['EPS RS'] > 200)", "& (df['EPS RS'] > 500)", ""]
cond2 = "& (df['Rev RS'] > 0)" # @param ["& (df['Rev RS'] > 0)", "& (df['Rev RS'] > 5)", "& (df['Rev RS'] > 10)", "& (df['Rev RS'] > 20)", "& (df['Rev RS'] > 50)", "& (df['Rev RS'] > 100)", ""]
cond3 = "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))" # @param ["& (df['TTM EPS'] > 0)", "& (df['TTM RPS'] > 0)", "& ((df['TTM EPS'] > 0) & (df['TTM RPS'] > 0))", ""]
cond4 = "& (df['EPS YoY (%)'] > 0)" # @param ["& (df['EPS YoY (%)'] > 0)", "& ((df['EPS YoY (%)'] > 0) & (df['YoY 2Q Algo (%)'] > 0))", ""]
cond5 = "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))" # @param ["& (df['EPS QoQ (%)'] > 0)", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0))", "& ((df['EPS QoQ (%)'] > 0) & (df['QoQ 2Q Algo (%)'] > 0) & (df['QoQ 3Q Algo (%)'] > 0))", ""]
cond6 = "" # @param ["& (df['Price'] < 50)", "& (df['Price'] < 100)", "& (df['Price'] < 200)", ""]
cond7 = "" # @param ["& (df['Sector'] == 'Technology')", "& (df['Sector'] == 'Energy')", ""]
sorted_column = "EPS RS" # @param ["EPS RS", "Rev RS"]
ascending = False # @param {"type":"boolean"}
num_items = 10 # @param [10, 20, 30] {"type":"raw"}

NUM_CONDS = 8
cond = eval('+'.join([f"cond{i}" for i in range(NUM_CONDS)]))

df = df_fin.copy()

import numpy as np

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df = df[eval(cond)]
df = df.sort_values(by=sorted_column, ascending=ascending)
df_top_f5 = df.head(num_items)
display(df_top_f5)

print_column(df_top_f5, 'Name')
print_column(df_top_f5, 'Ticker')

,Ticker,Sector,Industry,Price,EPS QoQ (%),QoQ 2Q Algo (%),QoQ 3Q Algo (%),EPS YoY (%),YoY 2Q Algo (%),EPS RS,TTM EPS,Rev RS,TTM RPS,TTM PE,Rating (EPS RS),Rating (Rev RS)
14,NXT,Technology,Solar,35.240,0.12,0.14,0.05,0.95,37.25,22225862.16,3.78,3.083000e+01,27.133,8.98,99.0,87.0
38,CRKN,Basic Materials,Specialty Chemicals,1.170,2.31,3.12,0.76,460.85,36.54,16858.10,3117.84,5.888889e+13,10.230,0.00,98.0,96.0
62,CALM,Consumer Defensive,Farm Products,89.900,0.01,0.01,7.60,153.00,-0.09,6049.15,8.73,4.365000e+01,54.437,10.28,98.0,89.0
66,NXU,Industrials,Electrical Equipment & Parts,0.252,0.65,17.63,0.95,109.20,54.10,5681.13,72.45,8.733333e+14,0.097,0.00,98.0,96.0
70,FFIE,Consumer Cyclical,Auto Manufacturers,2.450,1.13,8.09,0.89,79.81,72.06,5302.38,866.40,2.824444e+15,0.378,0.00,98.0,97.0
76,LTH,Consumer Cyclical,Leisure,24.970,1.08,0.53,1.12,2.00,-0.08,4808.29,0.53,1.159000e+01,12.210,46.66,97.0,79.0
106,LNW,Consumer Cyclical,Gambling,96.080,0.01,0.05,0.05,93.00,2.79,3264.42,3.31,2.570000e+00,34.000,28.71,97.0,70.0
236,SEZL,Financial Services,Credit Services,156.240,2.72,0.72,2.57,25.25,3.55,1202.40,7.05,4.517000e+01,33.982,22.63,94.0,89.0
238,EFC,Real Estate,REIT - Mortgage,12.420,0.94,0.78,0.80,14.50,NaN,1198.88,1.21,8.809000e+01,3.625,10.28,94.0,91.0
270,EGO,Basic Materials,Gold,16.550,0.59,1.62,2.62,26.00,0.70,1072.83,0.88,5.400000e+00,5.453,19.29,93.0,73.0


NXT, CRKN, CALM, NXU, FFIE, LTH, LNW, SEZL, EFC, EGO


## Step 3. Visualize Filtered Stocks

In [30]:
# @title Plot 1. Mansfield Relative Strength Comparison {"run":"auto"}
source = "Filter 5. Financial" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout", "Filter 4. Groupby Industry", "Filter 5. Financial", "Major Global Stock Indices", "Sector Indices"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

MAX_STOCK_SELECTION = 10
N_COLS = 8  # the number of columns of the grid layout for checkboxes

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Filter 5. Financial': lambda: tickers_from_df(df_top_f5),
    'Major Global Stock Indices': major_indices,
    'Sector Indices': sector_indices,
}[source]()

checkboxes = cbs_create(symbols, MAX_STOCK_SELECTION)
checkbox_grid = cbs_with_grid(checkboxes, N_COLS)
btn_unselect_all = widgets.Button(description="Unselect All")
btn_unselect_all.on_click(lambda b: cbs_unselect_all(checkboxes))
btn_select_top = widgets.Button(description="Select Top 10")
btn_select_top.on_click(lambda b: cbs_select_top(checkboxes, MAX_STOCK_SELECTION))

period_dropdown = create_period_dropdown(metadata['Period'])
interval_dropown = create_interval_dropdown()

cmp_theme_dropdown = {
    'mplfinance': create_style_dropdown('Comparison Theme:', 'charles'),
    'Plotly': create_template_dropdown('Comparison Theme:', 'plotly_dark'),
}[backend]
btn_plot = widgets.Button(description="Generate Plot")

out_msg, out_fig = widgets.Output(), widgets.Output()

ui = widgets.VBox([
    checkbox_grid,
    widgets.HBox([btn_unselect_all, btn_select_top]),
    widgets.VBox([period_dropdown, interval_dropown]),
    widgets.VBox([cmp_theme_dropdown, btn_plot]),
    out_msg, out_fig
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

rs_cmp = {
    'mplfinance': mpl_mansfield.RelativeStrengthLines,
    'Plotly': ply_mansfield.RelativeStrengthLines,
}[backend]

def on_checkbox_change(change):
    selected_count = sum([cb.value for cb in checkboxes])
    if selected_count > MAX_STOCK_SELECTION:
        # Uncheck the last checked box if selection exceeds limit
        changed_checkbox = change['owner']
        changed_checkbox.value = False
        with out_msg:
            out_fig.clear_output()
            print(f"Only {MAX_STOCK_SELECTION} stocks can be selected at most.")

# Bind the checkbox change event to the function
for checkbox in checkboxes:
    checkbox.observe(on_checkbox_change, names='value')

def on_plot_click(b):
    symbols = cbs_get_selected(checkboxes)
    if not symbols:
        with out_msg:
            out_fig.clear_output()
            print("No stocks selected. Please select at least one stock.")
        return
    with out_fig:
        out_msg.clear_output()
        clear_output()
        interval = interval_dropown.value
        period = period_dropdown.value
        if rs_cmp is mpl_mansfield.RelativeStrengthLines:
            rs_cmp.plot(symbols, interval=interval, period=period,
                        style=cmp_theme_dropdown.value,
                        color_cycle=plt.cm.Paired.colors)
        else: # Plotly
            rs_cmp.plot(symbols, interval=interval, period=period,
                        template=cmp_theme_dropdown.value,
                        colorway=px.colors.qualitative.Set3)

btn_plot.on_click(on_plot_click)

if backend == 'Plotly':
    enable_plotly_in_cell()


In [35]:
# @title Plot 2. Mansfield Stock Chart {"run":"auto"}
source = "Filter 5. Financial" # @param ["Filter 1. Sorting", "Filter 2. Increasing RS","Filter 3. RS Breakout","Filter 4. Groupby Industry","Filter 5. Financial","Major Global Stock Indices", "U.S. Listed Stocks", "Taiwan Stocks"]
backend = "mplfinance" # @param ["mplfinance","Plotly"]

import functools as ft
import ipywidgets as widgets
from IPython.display import display, clear_output
import yfinance as yf

import vistock.stock_indices as si
import vistock.tw as tw

symbols = {
    'Filter 1. Sorting': lambda: tickers_from_df(df_top_f1),
    'Filter 2. Increasing RS': lambda: tickers_from_df(df_top_f2),
    'Filter 3. RS Breakout': lambda: tickers_from_df(df_top_f3),
    'Filter 4. Groupby Industry': lambda: tickers_from_df(df_top_f4),
    'Filter 5. Financial': lambda: tickers_from_df(df_top_f5),
    'Major Global Stock Indices': major_indices,
    'U.S. Listed Stocks': ft.partial(si.get_tickers, 'USLS'),
    'Taiwan Stocks': ft.partial(si.get_tickers, 'TWSE+TPEX+ESB'),
}[source]()
if source == 'Taiwan Stocks':
    symbols = [f"{tw_stock_name(s)} {s}" for s in symbols]

dropdowns, layout = create_search_dropdowns(symbols, 1)

period_dropdown2 = create_period_dropdown(metadata['Period'])
interval_dropown2 = create_interval_dropdown('1wk')
stock_theme_dropdown = {
    'mplfinance': create_style_dropdown('Stock Theme:', 'yahoo'),
    'Plotly': create_template_dropdown('Stock Theme:', 'plotly'),
}[backend]

btn_plot_prc = widgets.Button(description="Price/RS/Volume Chart",
                              layout=widgets.Layout(width='168px'))
btn_plot_fin = widgets.Button(description="Financial Chart",
                              layout=widgets.Layout(width='168px'))
btn_report_q = widgets.Button(description="Quarterly Report",
                              layout=widgets.Layout(width='168px'))
btn_report_a = widgets.Button(description="Annual Report",
                              layout=widgets.Layout(width='168px'))
btn_clear_last = widgets.Button(description="Clear Last")
btn_clear_all = widgets.Button(description="Clear All")
#outputs = widgets.VBox()

ui = widgets.VBox([
    layout,
    period_dropdown2, interval_dropown2, stock_theme_dropdown,
    widgets.HBox([btn_plot_prc, btn_plot_fin]),
    widgets.HBox([btn_report_q, btn_report_a]),
    widgets.HBox([btn_clear_last, btn_clear_all]),
    outputs
])
display(ui)

import vistock.mpl.mansfield as mpl_mansfield
import vistock.plotly.mansfield as ply_mansfield

stock_chart = {
    'mplfinance': mpl_mansfield.StockChart,
    'Plotly': ply_mansfield.StockChart,
}[backend]

def get_symbols():
    symbols = get_dropdowns_selected_options(dropdowns)
    if source == 'Taiwan Stocks':
        symbols = [s.split()[0] for s in symbols]
    return symbols

def on_plot_prc_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        interval = interval_dropown2.value
        period = period_dropdown2.value
        if not selected:
            print("No Stock Selected!")
        elif stock_chart is mpl_mansfield.StockChart:
            stock_chart.plot(selected[0], interval=interval, period=period,
                             style=stock_theme_dropdown.value,
                             legend_loc='upper left')
        else: # Plotly
            stock_chart.plot(selected[0], interval=interval, period=period,
                             template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

import vistock.mpl.financials as mpl_fin
import vistock.plotly.financials as ply_fin

fin_chart = {
    'mplfinance': mpl_fin,
    'Plotly': ply_fin,
}[backend]

def on_plot_fin_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        elif fin_chart is mpl_fin:
            fin_chart.plot(selected[0], style=stock_theme_dropdown.value)
        else: # Plotly
            fin_chart.plot(selected[0], template=stock_theme_dropdown.value)
    outputs.children = (new_output,) + outputs.children

def on_report_q_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Quarterly Financials:")
            display(yf.Ticker(ticker).quarterly_financials)
    outputs.children = (new_output,) + outputs.children

def on_report_a_click(b):
    selected = get_symbols()
    new_output = widgets.Output()
    with new_output:
        if not selected:
            print("No Stock Selected!")
        else:
            symbol = tw.as_yfinance(selected[0])
            ticker = tw.as_yfinance(symbol)
            print(f"\n{symbol} Annual Financials:")
            display(yf.Ticker(ticker).financials)
    outputs.children = (new_output,) + outputs.children

def on_clear_last_click(b):
    if outputs.children:
        children = list(outputs.children)
        children.pop(0)
        outputs.children = tuple(children)

def on_clear_all_click(b):
    outputs.children = ()

btn_plot_prc.on_click(on_plot_prc_click)
btn_plot_fin.on_click(on_plot_fin_click)
btn_report_q.on_click(on_report_q_click)
btn_report_a.on_click(on_report_a_click)
btn_clear_last.on_click(on_clear_last_click)
btn_clear_all.on_click(on_clear_all_click)

if backend == 'Plotly':
    enable_plotly_in_cell()
